In [0]:
from pyspark.sql.functions import col, isnan, when, count, sum, min, max, countDistinct, month, year

In [0]:
# Nome da database e tabela
database_name = "PRATA"
table_name = "2013_hard_drive_csv_parquet_prata"

df = spark.table(f"{database_name}.{table_name}")
df.limit(10).display()

_c0,date,serial_number,model,capacity_bytes,failure,Read_Error_Rate,Reallocated_Sectors_Count,Power_On_Hours,Temperature,Current_Pending_Sector_Count
4329235,2013-12-03,6XW05AYQ,ST31500541AS,1500301910016,false,40572143,0,36962,23,0
4329236,2013-12-03,WD-WCAWZ0827930,WDC WD30EZRX,3000592982016,false,0,0,12830,21,0
4329237,2013-12-03,MJ1311YNG3EJJA,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,18736,26,0
4329238,2013-12-03,13H32WEAS,TOSHIBA DT01ACA300,3000592982016,false,0,0,5313,30,0
4329239,2013-12-03,PL1321LAG31Y7H,Hitachi HDS5C4040ALE630,4000787030016,false,0,0,6460,23,0
4329240,2013-12-03,6XW05YMX,ST31500541AS,1500301910016,false,83783737,0,36962,24,0
4329241,2013-12-03,MK0311YHG9JPAA,Hitachi HDS723030ALA640,3000592982016,false,131077,17,16826,28,0
4329242,2013-12-03,MJ1311YNG6KX6A,Hitachi HDS5C3030ALA630,3000592982016,false,0,0,14493,23,0
4329243,2013-12-03,Z300CR9R,ST4000DM000,4000787030016,false,24949392,0,2651,20,0
4329244,2013-12-03,W300B2TV,ST4000DM000,4000787030016,false,60690512,0,3395,17,0


Os servidores são confiáveis?

In [0]:
# Agrupar pela coluna 'failure' e contar as ocorrências
result_failure = df.groupBy("failure").count()
result_failure.show()



+-------+-------+
|failure|  count|
+-------+-------+
|   true|    724|
|  false|5090777|
+-------+-------+



No contexto da confiabildiade dos servidores, pela diferença entre a quantidade de falhas e de não falhas, se pode ver que os servidores são equipamentos com baixa taxa de falha e alta disponibilidade.

Realizando as filtragens para trabalhar somente com as falhas

In [0]:
# Filtrar os valores 'failure' com valor True
df_filtered = df.filter(df["failure"] == True)

Quais os modelos que mais falham?

In [0]:
# Agrupar pela coluna modelo e contar as ocorrências
result_model = df_filtered.groupBy("model").count().orderBy("count", ascending=False)
result_model.show()


+--------------------+-----+
|               model|count|
+--------------------+-----+
|         ST3000DM001|  254|
|        ST31500541AS|  123|
|        ST31500341AS|   91|
|         ST1500DL003|   51|
|         ST4000DM000|   48|
|Hitachi HDS722020...|   28|
|Hitachi HDS5C3030...|   26|
|Hitachi HDS5C4040...|   23|
|        WDC WD30EZRX|   14|
|        ST32000542AS|   14|
|        ST33000651AS|   12|
|        WDC WD10EADS|   12|
|         ST2000DL003|    7|
|Hitachi HDS723030...|    6|
|         ST2000DL001|    4|
|        WDC WD30EFRX|    4|
|  TOSHIBA DT01ACA300|    2|
|        ST320005XXXX|    1|
|Hitachi HDS723020...|    1|
|HGST HMS5C4040ALE640|    1|
+--------------------+-----+
only showing top 20 rows



No resultado vemos o modelo com mais falhas no período analisado, evidente que para se ter mais credibilidade na análise um levantamento de mais dados levando em conta quantiades seria necessária.

Qual a distribuição de itens em relação aos modelos de servidores?

In [0]:
# Agrupar pela coluna modelo e contar os diferentes serial_number
result_model_p_serial = df.groupBy("model").agg(countDistinct("serial_number").alias("distinct_serial_number_count")).orderBy("distinct_serial_number_count", ascending=False)

result_model_p_serial.show()

+--------------------+----------------------------+
|               model|distinct_serial_number_count|
+--------------------+----------------------------+
|         ST4000DM000|                        5525|
|Hitachi HDS722020...|                        4753|
|         ST3000DM001|                        4658|
|Hitachi HDS5C3030...|                        4621|
|Hitachi HDS5C4040...|                        2703|
|        ST31500541AS|                        2187|
|Hitachi HDS723030...|                        1034|
|        ST31500341AS|                         785|
|        WDC WD10EADS|                         550|
|        WDC WD30EZRX|                         489|
|        ST32000542AS|                         381|
|        WDC WD30EFRX|                         355|
|        ST33000651AS|                         309|
|         ST4000DX000|                         180|
|         ST1500DL003|                         116|
|        WDC WD10EACS|                         109|
|  TOSHIBA D

Alguns modelos de discos rígidos, como o ST4000DM000, Hitachi HDS722020ALA330, e ST3000DM001, têm um número significativamente maior de serial_number distintos, indicando que esses modelos são amplamente utilizados.
Por outro lado, modelos como o WDC WD10EADX e HGST HDS724040ALE640 têm um número muito menor de serial_number distintos, sugerindo que esses modelos são menos comuns.
A tabela mostra que modelos das marcas Seagate (ST) e Hitachi são muito populares, com múltiplos modelos listados e um grande número de serial_number distintos.
Modelos da Western Digital (WDC) também aparecem na lista, mas com um menor número de serial_number distintos em comparação com Seagate e Hitachi.
A presença de diferentes modelos com variados números de serial_number distintos sugere que existe uma variedade de capacidades e tipos de discos rígidos. Por exemplo, modelos com alta capacidade de armazenamento podem ter um número maior de serial_number distintos devido à sua popularidade em ambientes de grandes volumes de dados.

Considerando o ano de 2013 dos dados, em quais meses mais falhas ocorrem?

In [0]:
# Adicionar colunas de mês
df_filtered = df_filtered.withColumn("month", month("date"))

# Agrupar pelas colunas 'month' e contar as ocorrências de 'failure'
result_failure_p_month = df_filtered.groupBy("month").agg(count("failure").alias("failure_count")).orderBy("failure_count", ascending = False)

# Mostrar os resultados
result_failure_p_month.show()

+-----+-------------+
|month|failure_count|
+-----+-------------+
|   12|          131|
|   11|          124|
|    5|          119|
|    7|          104|
|    6|           81|
|   10|           69|
|    8|           49|
|    4|           43|
|    9|            4|
+-----+-------------+



Na questão de falha em relação ao mês, se nota que os dois últimos meses do ano possuem maior concentração de falhas. Com valores semelhantes se tem os meses 5 e 7, uma análise relacionada aos fatores climáticos e de utilização dos mesmo, seria interessante. 

In [0]:


# Agrupar pela coluna 'Temperature' e contar os valores 'failure' com valor True
result = df_filtered.groupBy("Temperature").count().orderBy("count", ascending = False)

# Mostrar os resultados
result.show()

+-----------+-----+
|Temperature|count|
+-----------+-----+
|         23|   93|
|         22|   80|
|         24|   76|
|         25|   65|
|         26|   64|
|         21|   57|
|         27|   54|
|         20|   43|
|         28|   40|
|         19|   39|
|         29|   36|
|         30|   18|
|         18|   17|
|         17|   12|
|         31|    8|
|         32|    7|
|         33|    5|
|         34|    3|
|         36|    3|
|         35|    3|
+-----------+-----+
only showing top 20 rows



Ao contrário do que seris esperado não são em temperaturas mais altas a maior ocorrência de falhas. Na faixa dos 20 e 30 graus se encontram a maioria das falhas, mais informações seriam necesárias para identificar a razão dessa concentração.

A utilização do Apache Spark para a análise dos dados de discos rígidos mostrou-se extremamente vantajosa, especialmente ao considerar a capacidade de processamento de grandes volumes de informações. Em uma sprint anterior, devido às limitações da plataforma utilizada, foi possível trabalhar apenas com uma versão reduzida deste dataset. No entanto, com o uso do Spark, foi possível carregar e analisar um volume de dados significativamente maior, permitindo uma análise mais abrangente e detalhada.

Os resultados obtidos até o momento indicam a popularidade e a distribuição dos diferentes modelos de discos rígidos, bem como a variedade de capacidades disponíveis. Modelos como ST3000DM001, ST31500541AS e ST4000DM000 destacaram-se por possuírem um grande número de ocorrências e serial_number distintos, sugerindo uma alta demanda e utilização desses modelos. Por outro lado, modelos com um menor número de ocorrências, como HGST HMS5C4040ALE640 e Hitachi HDS723020BLA642, podem ser considerados de nicho ou potencialmente descontinuados.

Para uma análise mais aprofundada e conclusões assertivas sobre os modelos e sazonalidades, seria necessário dispor de informações adicionais, como dados temporais, taxas de falhas e detalhes de uso. Esses dados complementares permitiriam identificar tendências ao longo do tempo, analisar a confiabilidade dos modelos e fornecer insights sobre os segmentos de mercado atendidos por cada fabricante.

O uso das ferramentas e aprendizados adquiridos durante este trabalho tem um imenso potencial de aplicação na atividade profissional. A simplicidade e a eficiência do Spark permitirão avanços significativos na análise de grandes volumes de dados, oferecendo uma base sólida para a tomada de decisões informadas e estratégicas.

Em resumo, a capacidade do Spark de lidar com grandes quantidades de dados de forma eficiente abre novas possibilidades para análises detalhadas e abrangentes, contribuindo significativamente para a melhoria contínua dos processos e resultados organizacionais
